In [ ]:
!pip install python-telegram-bot --upgrade
!pip install dextools-python

In [27]:
api_key = "b5ZO15hnCV9dnJr7XOQZIaGMWXCb6y5J7bXSoJvS"

In [28]:
from dextools_python import DextoolsAPIV2
import pandas as pd
import time
import datetime

dextools = DextoolsAPIV2(api_key, plan="trial")

In [29]:
# Get Tokens

In [30]:
def get_liquidity_pools_sorted_by_date_df(chain='solana', start_date=None, end_date=None, num_pages=3):
    if not start_date:
        start_date = datetime.datetime.now()
    if not end_date:
        end_date = start_date - datetime.timedelta(hours=6)
    
    start_date_str = start_date.strftime('%Y-%m-%dT%H:%M:%S')
    end_date_str = end_date.strftime('%Y-%m-%dT%H:%M:%S')
    
    all_tokens = []
    for page in range(0, num_pages):
        response = dextools.get_pools(chain, from_=end_date_str, to=start_date_str, page=page, order="desc")
        print(response)
        all_tokens.extend(response["data"]['results'])
        time.sleep(1.5)
    
    # Flatten the nested 'socialInfo' and convert data to DataFrame
    tokens_df = pd.json_normalize(all_tokens)
    
    return tokens_df

In [31]:
def run_get_tokens():
    tokens = get_liquidity_pools_sorted_by_date_df()
    return tokens

In [33]:
tokens = run_get_tokens()
"Complete"

{'statusCode': 200, 'data': {'page': 0, 'pageSize': 20, 'totalPages': 24, 'results': [{'creationBlock': 259313707, 'creationTime': '2024-04-10T06:01:22.746Z', 'exchange': {'name': 'Meteora', 'factory': 'eo7wjkq67rjjqszxs6z3ykapzy3emj6xy8x5eqvn5uab'}, 'address': 'Ce7Jux9kzDXrGCdPG2sh6PkR5FyHa8fjZPdacdJbfkio', 'mainToken': {'name': 'BaDonkaDonk Inu', 'symbol': 'BADONK', 'address': 'HWQbBo1WyrEfXiAcDXdSE86k4wnhwut9HoaWujGeo63y'}, 'sideToken': {'name': 'Wrapped SOL', 'symbol': 'SOL', 'address': 'So11111111111111111111111111111111111111112'}}, {'creationBlock': 259313643, 'creationTime': '2024-04-10T06:00:44.491Z', 'exchange': {'name': 'Raydium', 'factory': '675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8'}, 'address': '7zTfQFHqoQR9vadFswjaE4nUBFTiGcgZq8D3SJYAJfH', 'mainToken': {'name': 'Trumpwifhat', 'symbol': 'TRUMP', 'address': '3eeoXuNMmNDdCCyhKBvUVVz79GwpHAH9zbVFf4a1iYdG'}, 'sideToken': {'name': 'Wrapped SOL', 'symbol': 'SOL', 'address': 'So11111111111111111111111111111111111111112'}}, {'

'Complete'

In [34]:
display(tokens)

,creationBlock,creationTime,address,exchange.name,exchange.factory,mainToken.name,mainToken.symbol,mainToken.address,sideToken.name,sideToken.symbol,sideToken.address
0,259313707.0,2024-04-10T06:01:22.746Z,Ce7Jux9kzDXrGCdPG2sh6PkR5FyHa8fjZPdacdJbfkio,Meteora,eo7wjkq67rjjqszxs6z3ykapzy3emj6xy8x5eqvn5uab,BaDonkaDonk Inu,BADONK,HWQbBo1WyrEfXiAcDXdSE86k4wnhwut9HoaWujGeo63y,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
1,259313643.0,2024-04-10T06:00:44.491Z,7zTfQFHqoQR9vadFswjaE4nUBFTiGcgZq8D3SJYAJfH,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,Trumpwifhat,TRUMP,3eeoXuNMmNDdCCyhKBvUVVz79GwpHAH9zbVFf4a1iYdG,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
2,259313384.0,2024-04-10T05:58:31.152Z,GZS4Hh6jLjC8U2isFooZDnQzTgTvxZEczMFMfrMLTPXQ,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,Thug Cat,THUGCAT,2PqPr9pYNHBs6BHDaSBnrwJqQjjv8PguK5rpQ5BqR71g,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
3,259312938.0,2024-04-10T05:54:50.586Z,8pzqhcXi6k6wtoLntUj8Ke7tFu31dUP1rNLvRp6jX2Fs,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,lol,dsfs,E1gQYsijkxLEQ7ZUTCjus1bb6yWDAcEYY6TksTvMUjae,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
4,259312812.0,2024-04-10T05:53:45.741Z,8AF5KsTnLqUE32WB9QXZFMCogmPASyFuj8tTc3mEbxhz,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,MemeMasterAI,MEME,9CAWv9wCdgucvQsAhqzTE1dtAG3dzUbYP8EkbaVtDiRG,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
5,259312788.0,2024-04-10T05:53:31.228Z,6StuQBCAFeSXfF1vWJFJA8KaS44Vm1BykSW6Y1VDDg2Q,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,Donald Trump,PUMP,43XHyZGbFAtXu3sEUFMi4XprFno68MLgtR3pkZk7weXB,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
6,259312716.0,2024-04-10T05:52:50.505Z,E1gDmS9xRANWjUYiBbxQ5B9hV5wN4Xyhqq8CLxZWcDEQ,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,GizmoByte,GZBY,ERPq6vfM67d5cyphYiTao1hJ46P8DqF11Dtntaj251LR,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
7,259312680.0,2024-04-10T05:52:42.576Z,AJZQKhcerTAKYchYSvSZvJkmw3d2hA36jCkopYHxvxmJ,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,Galactic Gazelle,GGLZ,7b2buJXLoS1H6GSzWLadAXrHaMxDuRQhJqHm5n8wTL6T,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
8,259312660.0,2024-04-10T05:52:28.885Z,FfXoiCdP2KhCn3sHc3hcM2wScgmnXHCg89NRDrxAT4og,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,Solmilio,Solmilio,4HqQLx6QpAdK3qFdB7eu4R1huitkJpqr7aGwkjkRmY5g,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
9,259312480.0,2024-04-10T05:50:55.403Z,DrPN23CLLVA6j8GiRNsafQAs2SdGYYXsMuqMCUouk2zy,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,Ansem Z Cat,Z Cat,zcAtp345gNzgELH6qqk5Ctx4LgyChyWr2Nh5nM3d6jH,Wrapped SOL,SOL,So11111111111111111111111111111111111111112


In [35]:
import requests

def get_rug_check_data(token_address):
    # Base API URL
    base_url = 'https://api.rugcheck.xyz/v1/tokens/'
    
    # Complete API URL with the token address
    api_url = f'{base_url}{token_address}/report'
    
    # Send GET request
    response = requests.get(api_url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response into a pandas DataFrame
        data = response.json()
        df_top_holders = pd.json_normalize(data)
        return df_top_holders
    else:
        return f"Failed to fetch data: status code {response.status_code}"

In [36]:
# Test Get Rug Check Data
token_address = '3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43'
df = get_rug_check_data(token_address)

In [37]:
pd.set_option('display.max_columns', None)
display(df)

,mint,tokenProgram,token_extensions,topHolders,freezeAuthority,mintAuthority,risks,score,lockerOwners,lpLockers,markets,rugged,tokenType,token.mintAuthority,token.supply,token.decimals,token.isInitialized,token.freezeAuthority,tokenMeta.name,tokenMeta.symbol,tokenMeta.uri,tokenMeta.mutable,tokenMeta.updateAuthority,fileMeta.description,fileMeta.name,fileMeta.symbol,fileMeta.image,transferFee.pct,transferFee.maxAmount,transferFee.authority
0,3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,None,[{'address': 'DyWNa439guZzgca6GFwhHgFjr8iGm9Vq...,None,None,"[{'name': 'Low amount of LP Providers', 'value...",100,None,None,[{'pubkey': '7XQh6FNHiumtgjCXPiDiSnDWoEGd9ZYHr...,False,,None,999996749143392774,9,True,None,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmPU63r9RMDM35w...,False,EKJCvZtJh9YMc4TycxfatAUAC4XT6sXcw91wrnLWGegN,,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmeZaCHdmPSkqyj...,0,0,11111111111111111111111111111111


In [38]:
def add_top_10_holders_percentage(df):
    # Assuming df['topHolders'] is a string representation of the topHolders list
    # Convert the string to a list of dictionaries if needed
    if isinstance(df['topHolders'].iloc[0], str):
        df['topHolders'] = df['topHolders'].apply(eval)

    # Normalize the nested JSON data for topHolders and calculate the sum of the 'pct' of the top 10 holders
    top_holders_df = pd.json_normalize(df['topHolders'].iloc[0])
    top_10_holders_pct = top_holders_df.nlargest(10, 'pct')['pct'].sum()

    # Add the top 10 holders percentage to the DataFrame
    df['top_10_holders_percentage'] = top_10_holders_pct

    # Drop the original topHolders column if you want to remove it
    #df.drop(columns=['topHolders'], inplace=True)

    return df

In [39]:
df = add_top_10_holders_percentage(df)

In [40]:
df

,mint,tokenProgram,token_extensions,topHolders,freezeAuthority,mintAuthority,risks,score,lockerOwners,lpLockers,markets,rugged,tokenType,token.mintAuthority,token.supply,token.decimals,token.isInitialized,token.freezeAuthority,tokenMeta.name,tokenMeta.symbol,tokenMeta.uri,tokenMeta.mutable,tokenMeta.updateAuthority,fileMeta.description,fileMeta.name,fileMeta.symbol,fileMeta.image,transferFee.pct,transferFee.maxAmount,transferFee.authority,top_10_holders_percentage
0,3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,None,[{'address': 'DyWNa439guZzgca6GFwhHgFjr8iGm9Vq...,None,None,"[{'name': 'Low amount of LP Providers', 'value...",100,None,None,[{'pubkey': '7XQh6FNHiumtgjCXPiDiSnDWoEGd9ZYHr...,False,,None,999996749143392774,9,True,None,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmPU63r9RMDM35w...,False,EKJCvZtJh9YMc4TycxfatAUAC4XT6sXcw91wrnLWGegN,,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmeZaCHdmPSkqyj...,0,0,11111111111111111111111111111111,60.049871


In [41]:
import pandas as pd
import ast
import json

def calculate_market_values(df):
    # Convert string representation of lists back into actual lists for the 'topHolders' column
    df['topHolders'] = df['topHolders'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    
    # Initialize total percentage and liquidity
    total_pct = 0
    liquidity_usd = 0

    # Iterate over each row for topHolders to sum percentages and calculate liquidity
    for index, row in df.iterrows():
        # Calculate total percentage from topHolders
        top_holders = row['topHolders']
        total_pct += sum(holder['pct'] for holder in top_holders)
        
        # Calculate liquidity from markets (assuming 'markets' is a stringified JSON in the DataFrame)
        markets_data = json.loads(row['markets']) if isinstance(row['markets'], str) else row['markets']
        for market in markets_data:
            liquidity_usd += market['lp']['baseUSD'] + market['lp']['quoteUSD']

    
    
    # Add the calculated values as new columns to the DataFrame
    df['total_percentage_of_top_holders'] = total_pct
    df['total_liquidity_usd'] = liquidity_usd
    return df

In [42]:
df = calculate_market_values(df)

display(df)

,mint,tokenProgram,token_extensions,topHolders,freezeAuthority,mintAuthority,risks,score,lockerOwners,lpLockers,markets,rugged,tokenType,token.mintAuthority,token.supply,token.decimals,token.isInitialized,token.freezeAuthority,tokenMeta.name,tokenMeta.symbol,tokenMeta.uri,tokenMeta.mutable,tokenMeta.updateAuthority,fileMeta.description,fileMeta.name,fileMeta.symbol,fileMeta.image,transferFee.pct,transferFee.maxAmount,transferFee.authority,top_10_holders_percentage,total_percentage_of_top_holders,total_liquidity_usd
0,3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,None,[{'address': 'DyWNa439guZzgca6GFwhHgFjr8iGm9Vq...,None,None,"[{'name': 'Low amount of LP Providers', 'value...",100,None,None,[{'pubkey': '7XQh6FNHiumtgjCXPiDiSnDWoEGd9ZYHr...,False,,None,999996749143392774,9,True,None,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmPU63r9RMDM35w...,False,EKJCvZtJh9YMc4TycxfatAUAC4XT6sXcw91wrnLWGegN,,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmeZaCHdmPSkqyj...,0,0,11111111111111111111111111111111,60.049871,66.034209,1.011808e+06


In [43]:
def add_top_holders_percentage(df):
    # Assuming the 'topHolders' column is a list of dictionaries with the 'pct' key
    for index, row in df.iterrows():
        # Sort the top holders by percentage in descending order
        sorted_holders = sorted(row['topHolders'], key=lambda x: x['pct'], reverse=True)
        # Get the top ten holders
        top_ten_holders = sorted_holders[:10]
        for i, holder in enumerate(top_ten_holders):
            # Create a new column for each top holder's percentage
            df.at[index, f'top_holder_{i+1}_pct'] = holder['pct']

    return df

In [44]:
df = add_top_holders_percentage(df)
display(df)

,mint,tokenProgram,token_extensions,topHolders,freezeAuthority,mintAuthority,risks,score,lockerOwners,lpLockers,markets,rugged,tokenType,token.mintAuthority,token.supply,token.decimals,token.isInitialized,token.freezeAuthority,tokenMeta.name,tokenMeta.symbol,tokenMeta.uri,tokenMeta.mutable,tokenMeta.updateAuthority,fileMeta.description,fileMeta.name,fileMeta.symbol,fileMeta.image,transferFee.pct,transferFee.maxAmount,transferFee.authority,top_10_holders_percentage,total_percentage_of_top_holders,total_liquidity_usd,top_holder_1_pct,top_holder_2_pct,top_holder_3_pct,top_holder_4_pct,top_holder_5_pct,top_holder_6_pct,top_holder_7_pct,top_holder_8_pct,top_holder_9_pct,top_holder_10_pct
0,3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,None,[{'address': 'DyWNa439guZzgca6GFwhHgFjr8iGm9Vq...,None,None,"[{'name': 'Low amount of LP Providers', 'value...",100,None,None,[{'pubkey': '7XQh6FNHiumtgjCXPiDiSnDWoEGd9ZYHr...,False,,None,999996749143392774,9,True,None,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmPU63r9RMDM35w...,False,EKJCvZtJh9YMc4TycxfatAUAC4XT6sXcw91wrnLWGegN,,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmeZaCHdmPSkqyj...,0,0,11111111111111111111111111111111,60.049871,66.034209,1.011808e+06,41.082889,9.000029,2.307067,1.450862,1.446068,1.047744,1.000003,0.986643,0.887715,0.840852


In [45]:
import pandas as pd
import ast

def calculate_market_cap_from_df(df):
    # Initialize an empty list to store market cap results
    market_caps = []
    
    # Loop through each row in the DataFrame
    for index, row in df.iterrows():
        # Check if markets is a string and convert it to a list if necessary
        if isinstance(row['markets'], str):
            markets = ast.literal_eval(row['markets'])
        else:
            markets = row['markets']
        
        # Initialize total market cap for the row
        total_market_cap = 0

        # Loop through each market to calculate its market cap
        for market in markets:
            base_price = market['lp']['basePrice']  # Extract the base price for the market
            # Calculate market cap using total supply and base price
            market_cap = row['token.supply'] / (10 ** row['token.decimals']) * base_price  
            total_market_cap += market_cap  # Sum up the market cap for the row

        # Append the calculated market cap for the row to the list
        market_caps.append(total_market_cap)
    
    # Add the market cap list as a new column to the DataFrame
    df['market_cap'] = market_caps
    
    # Return the updated DataFrame
    return df

In [46]:
df = calculate_market_cap_from_df(df)

In [47]:
df

,mint,tokenProgram,token_extensions,topHolders,freezeAuthority,mintAuthority,risks,score,lockerOwners,lpLockers,markets,rugged,tokenType,token.mintAuthority,token.supply,token.decimals,token.isInitialized,token.freezeAuthority,tokenMeta.name,tokenMeta.symbol,tokenMeta.uri,tokenMeta.mutable,tokenMeta.updateAuthority,fileMeta.description,fileMeta.name,fileMeta.symbol,fileMeta.image,transferFee.pct,transferFee.maxAmount,transferFee.authority,top_10_holders_percentage,total_percentage_of_top_holders,total_liquidity_usd,top_holder_1_pct,top_holder_2_pct,top_holder_3_pct,top_holder_4_pct,top_holder_5_pct,top_holder_6_pct,top_holder_7_pct,top_holder_8_pct,top_holder_9_pct,top_holder_10_pct,market_cap
0,3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,None,[{'address': 'DyWNa439guZzgca6GFwhHgFjr8iGm9Vq...,None,None,"[{'name': 'Low amount of LP Providers', 'value...",100,None,None,[{'pubkey': '7XQh6FNHiumtgjCXPiDiSnDWoEGd9ZYHr...,False,,None,999996749143392774,9,True,None,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmPU63r9RMDM35w...,False,EKJCvZtJh9YMc4TycxfatAUAC4XT6sXcw91wrnLWGegN,,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmeZaCHdmPSkqyj...,0,0,11111111111111111111111111111111,60.049871,66.034209,1.011808e+06,41.082889,9.000029,2.307067,1.450862,1.446068,1.047744,1.000003,0.986643,0.887715,0.840852,1.229446e+06


In [65]:
dextools.get_token("solana", "3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43")

{'statusCode': 200,
 'data': {'address': '3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43',
  'name': 'Baby Slerf',
  'symbol': 'BABYSLERF',
  'decimals': 9,
  'socialInfo': {'bitbucket': '',
   'discord': '',
   'facebook': '',
   'github': '',
   'instagram': '',
   'linkedin': '',
   'medium': '',
   'reddit': '',
   'telegram': 'https://t.me/babyslerf_io',
   'tiktok': '',
   'twitter': 'https://twitter.com/babyslerf_io',
   'website': 'https://babyslerf.io/',
   'youtube': ''}}}

In [67]:
def update_social_info(df):
    # Create new columns with empty strings for each social info key
    social_keys = ['discord', 'facebook', 'github', 'instagram', 'linkedin', 
                   'medium', 'reddit', 'telegram', 'tiktok', 'twitter', 
                   'website', 'youtube', 'bitbucket']
    for key in social_keys:
        df[key] = ''

    # Iterate over the DataFrame and update with social info
    for index, row in df.iterrows():
        token_data = dextools.get_token('solana', row['mint'])
        social_info = token_data['data']['socialInfo']
        for key in social_keys:
            if key in social_info:
                df.at[index, key] = social_info[key]
    
    return df

In [68]:
df = update_social_info(df)

In [69]:
df

,mint,tokenProgram,token_extensions,topHolders,freezeAuthority,mintAuthority,risks,score,lockerOwners,lpLockers,markets,rugged,tokenType,token.mintAuthority,token.supply,token.decimals,token.isInitialized,token.freezeAuthority,tokenMeta.name,tokenMeta.symbol,tokenMeta.uri,tokenMeta.mutable,tokenMeta.updateAuthority,fileMeta.description,fileMeta.name,fileMeta.symbol,fileMeta.image,transferFee.pct,transferFee.maxAmount,transferFee.authority,top_10_holders_percentage,total_percentage_of_top_holders,total_liquidity_usd,top_holder_1_pct,top_holder_2_pct,top_holder_3_pct,top_holder_4_pct,top_holder_5_pct,top_holder_6_pct,top_holder_7_pct,top_holder_8_pct,top_holder_9_pct,top_holder_10_pct,market_cap,discord,facebook,github,instagram,linkedin,medium,reddit,telegram,tiktok,twitter,website,youtube,bitbucket
0,3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,None,[{'address': 'DyWNa439guZzgca6GFwhHgFjr8iGm9Vq...,None,None,"[{'name': 'Low amount of LP Providers', 'value...",100,None,None,[{'pubkey': '7XQh6FNHiumtgjCXPiDiSnDWoEGd9ZYHr...,False,,None,999996749143392774,9,True,None,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmPU63r9RMDM35w...,False,EKJCvZtJh9YMc4TycxfatAUAC4XT6sXcw91wrnLWGegN,,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmeZaCHdmPSkqyj...,0,0,11111111111111111111111111111111,60.049871,66.034209,1.011808e+06,41.082889,9.000029,2.307067,1.450862,1.446068,1.047744,1.000003,0.986643,0.887715,0.840852,1.229446e+06,,,,,,,,https://t.me/babyslerf_io,,https://twitter.com/babyslerf_io,https://babyslerf.io/,,
